In [ ]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import transformers
import torch
import pathlib
import awswrangler as wr
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role
from huggingface_hub import create_repo, upload_file, login
from tqdm.auto import tqdm
import fsspec


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


In [ ]:
login()

<<<<<<< local


>>>>>>> remote


In [8]:
_ = """
fs = fsspec.filesystem('s3')  # s3 / gcs / abfs / adl / oci / ...
subset = 'test'
data_dir = f's3://sagemaker-research-methodology-extraction/01_data/03_core/unified_works_{subset}/'
pattern = '*'
data_files = fs.glob(data_dir + pattern)

utils.ensure_path('temp_files')
# create_repo(destination_dataset, repo_type='dataset')
for data_file in tqdm(fs.glob(data_dir + pattern)):
    with fs.open(data_file, 'rb') as fileobj:
        path_in_repo = f'data/{subset}/{data_file[len(data_dir):]}'
        print(path_in_repo, type(fileobj))
        upload_file(
            path_or_fileobj=fileobj,
            path_in_repo=path_in_repo,
            repo_id=destination_dataset,
            repo_type="dataset",
        )
"""

In [9]:
_ = """
import smart_open

bucket = 'sagemaker-research-methodology-extraction'
subset = 'test'
data_dir = f's3://sagemaker-research-methodology-extraction/01_data/03_core/unified_works_{subset}/'
for key, content in s3.iter_bucket(bucket, prefix=data_dir, workers=8):
"""

In [10]:
import boto3
import os
from pathlib import Path
import time

s3_client = boto3.client('s3')
bucket_name = 'sagemaker-research-methodology-extraction'
timelogger = utils.TimeLogger()
destination_dataset = 'SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'
for subset in ['train', 'test', 'validation']:
    timelogger.log(f'START subset: {subset}')
    s3_folder_prefix = f'01_data/03_core/unified_works_{subset}'
    local_download_path = 'temp_files'
    utils.ensure_path(local_download_path)

    paginator = s3_client.get_paginator('list_objects_v2')
    file_entry_pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder_prefix)

    for file_entry_page in file_entry_pages:
        if 'Contents' in file_entry_page:
            for file_entry in file_entry_page['Contents']:
                s3_key = file_entry['Key']
                relative_path = Path(s3_key).relative_to(s3_folder_prefix)
                path_in_repo = f'data/{subset}/{relative_path}.parquet'
                print('path_in_repo', path_in_repo)
                local_file_path = Path(local_download_path) / subset / relative_path

                local_file_path.parent.mkdir(parents=True, exist_ok=True)
                # print('relative_path.resolve()', str(local_file_path.resolve()), type(local_file_path.resolve()))

                s3_client.download_file(bucket_name, s3_key, str(local_file_path))
                # print(f"{subset}: Downloaded {s3_key} to {local_file_path}")
                upload_result = upload_file(
                    path_or_fileobj=local_file_path,
                    path_in_repo=path_in_repo,
                    repo_id=destination_dataset,
                    repo_type="dataset",
                )
                # print(f"{subset}: Uploaded {s3_key} to {path_in_repo}")
                if '/home/sagemaker-user/research_methodology_extraction/src/04_transformation/temp_files/' in str(local_file_path.resolve()):
                    os.remove(local_file_path)
                    # print(f"{subset}: Deleted {s3_key} at {local_file_path}")
                time.sleep(10)
    timelogger.log(f'END subset: {subset}')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
 :: START subset: train | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
ensure_path(temp_files)
path_in_repo data/train/20250904_234024_00199_vpvv2_03ed3d45-472c-46a2-89d2-f53707dacbe5.parquet


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...3ed3d45-472c-46a2-89d2-f53707dacbe5:   0%|          |  524kB /  154MB            

path_in_repo data/train/20250904_234024_00199_vpvv2_04d8da34-f1d0-4570-9f40-9d5aef04cc2f.parquet


...

path_in_repo data/validation/20250904_234123_00159_rh4vh_f56b0dcb-febb-4f5b-a9f0-4c9f1c856e21.parquet


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...56b0dcb-febb-4f5b-a9f0-4c9f1c856e21:  23%|##2       | 3.67MB / 16.2MB            

 :: END subset: validation | since_start: 16.0 minutes, 1.87 seconds | since_last: 4.0 minutes, 47.17 seconds :: 


In [3]:
from datasets import load_dataset, DatasetDict, Dataset
dataset_name = 'SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'
dataset = load_dataset(dataset_name)
dataset

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

data/train/20250904_234024_00199_vpvv2_0(…):   0%|          | 0.00/154M [00:00<?, ?B/s]

data/train/20250904_234024_00199_vpvv2_0(…):   0%|          | 0.00/155M [00:00<?, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/22 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 346344
    })
    validation: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 43439
    })
    test: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 43478
    })
})

In [4]:
dataset_title_subfieldindex = dataset.select_columns(['title', 'subfield_index'])
dataset_title_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_Title_SubfieldIndex', num_proc=8)

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Title_SubfieldIndex/commit/2d2c92e9eab96c21183d65369bf545483ae05c50', commit_message='Upload dataset', commit_description='', oid='2d2c92e9eab96c21183d65369bf545483ae05c50', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Title_SubfieldIndex', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Title_SubfieldIndex'), pr_revision=None, pr_num=None)

In [5]:
dataset_title_topicindex = dataset.select_columns(['title', 'topic_index'])
dataset_title_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_Title_TopicIndex', num_proc=8)

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Title_TopicIndex/commit/40911f919c3308e0a6a8e6aa3e981dcc2178b99d', commit_message='Upload dataset', commit_description='', oid='40911f919c3308e0a6a8e6aa3e981dcc2178b99d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Title_TopicIndex', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Title_TopicIndex'), pr_revision=None, pr_num=None)

In [6]:
dataset_abstract_subfieldindex = dataset.select_columns(['abstract', 'subfield_index'])
dataset_abstract_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_Abstract_SubfieldIndex', num_proc=8)
dataset_abstract_topicindex = dataset.select_columns(['abstract', 'topic_index'])
dataset_abstract_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_Abstract_TopicIndex', num_proc=8)

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Abstract_TopicIndex/commit/e2f0fdcb967cb3052d25ccfc362e1d07a8a89304', commit_message='Upload dataset', commit_description='', oid='e2f0fdcb967cb3052d25ccfc362e1d07a8a89304', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Abstract_TopicIndex', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_Abstract_TopicIndex'), pr_revision=None, pr_num=None)

In [7]:
dataset_fulltext_subfieldindex = dataset.select_columns(['fulltext', 'subfield_index'])
dataset_fulltext_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_FullText_SubfieldIndex')

Uploading the dataset shards:   0%|          | 0/23 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   0%|          |  524kB /  222MB            

...

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   2%|1         | 3.67MB /  219MB            

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_FullText_SubfieldIndex/commit/b62ab3fadafe09db8d7beafeda6ef0a67387f62b', commit_message='Upload dataset', commit_description='', oid='b62ab3fadafe09db8d7beafeda6ef0a67387f62b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_FullText_SubfieldIndex', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_FullText_SubfieldIndex'), pr_revision=None, pr_num=None)

In [8]:
dataset_fulltext_topicindex = dataset.select_columns(['fulltext', 'topic_index'])
dataset_fulltext_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_FullText_TopicIndex')

Uploading the dataset shards:   0%|          | 0/23 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  15%|#4        | 33.2MB /  222MB            

...

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  19%|#9        | 41.7MB /  219MB            

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_FullText_TopicIndex/commit/200dc71ea5cd4b6560b22e4ff54ab6fdcb7df098', commit_message='Upload dataset', commit_description='', oid='200dc71ea5cd4b6560b22e4ff54ab6fdcb7df098', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_FullText_TopicIndex', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics_FullText_TopicIndex'), pr_revision=None, pr_num=None)